In [116]:
import xgboost as xgb

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from numba import jit
import math

import xgboost as xgb

from extract import extract_url

In [ ]:
def smape_fast(y_true, y_pred):
    out = 0
    for i in range(y_true.shape[0]):
        a = y_true[i]
        b = y_pred[i]
        c = a+b
        if c == 0:
            continue
        out += math.fabs(a - b) / c
    out *= (200.0 / y_true.shape[0])
    return out

In [70]:
def calculerLag(df,lag):
    df_lag = pd.DataFrame()
    for column in df.columns:
        df_column = pd.DataFrame(df[column])
        for i in range(lag):
            df_column['diff' + str(i+1)] = 0
            df_column['diff' + str(i+1)] = df_column[column].diff(i+1)
        
        df_column['Page'] = column
        df_column['Visiteurs'] = df_column[column]
        df_column = df_column.drop([column],axis= 1)
        df_lag.append(df_column)
        df_lag = pd.concat([df_lag, df_column], ignore_index=False)
    return(df_lag)

In [124]:
#Paramètres
reduction = 100
lag = 8

In [125]:
train_1 = pd.read_csv('../Data/train_1.csv')
train_1_reduce = train_1[0:reduction]
train_1_reduce = train_1_reduce.fillna(0)
df_extract = extract_url(train_1_reduce['Page'])
train_1_reduce = train_1_reduce.set_index('Page')
train_1_reduce = train_1_reduce.T.rename_axis('Dates')
df_lag = calculerLag(train_1_reduce,lag).reset_index()
data = df_lag.set_index('Page').join(df_extract.set_index('Page'))
data = data.reset_index().set_index(['Page','Dates'])

In [126]:
data.head(9)

diff1  diff2  diff3  \
Page                                    Dates                             
2NE1_zh.wikipedia.org_all-access_spider 2015-07-01    NaN    NaN    NaN   
                                        2015-07-02   -7.0    NaN    NaN   
                                        2015-07-03   -6.0  -13.0    NaN   
                                        2015-07-04    8.0    2.0   -5.0   
                                        2015-07-05    1.0    9.0    3.0   
                                        2015-07-06   -5.0   -4.0    4.0   
                                        2015-07-07    0.0   -5.0   -4.0   
                                        2015-07-08   13.0   13.0    8.0   
                                        2015-07-09    4.0   17.0   17.0   

                                                    diff4  diff5  diff6  \
Page                                    Dates                             
2NE1_zh.wikipedia.org_all-access_spider 2015-07-01    NaN    NaN    NaN   
                                        2015-07-02    NaN    NaN    NaN   
                                        2015-07-03    NaN    NaN    NaN   
                                        2015-07-04    NaN    NaN    NaN   
                                        2015-07-05   -4.0    NaN    NaN   
                                        2015-07-06   -2.0   -9.0    NaN   
                                        2015-07-07    4.0   -2.0   -9.0   
                                        2015-07-08    9.0   17.0   11.0   
                                        2015-07-09   12.0   13.0   21.0   

                                                    diff7  diff8  Visiteurs  \
Page                                    Dates                                 
2NE1_zh.wikipedia.org_all-access_spider 2015-07-01    NaN    NaN       18.0   
                                        2015-07-02    NaN    NaN       11.0   
                                        2015-07-03    NaN    NaN        5.0   
                                        2015-07-04    NaN    NaN       13.0   
                                        2015-07-05    NaN    NaN       14.0   
                                        2015-07-06    NaN    NaN        9.0   
                                        2015-07-07    NaN    NaN        9.0   
                                        2015-07-08    4.0    NaN       22.0   
                                        2015-07-09   15.0    8.0       26.0   

                                                                agent  \
Page                                    Dates                           
2NE1_zh.wikipedia.org_all-access_spider 2015-07-01  all-access_spider   
                                        2015-07-02  all-access_spider   
                                        2015-07-03  all-access_spider   
                                        2015-07-04  all-access_spider   
                                        2015-07-05  all-access_spider   
                                        2015-07-06  all-access_spider   
                                        2015-07-07  all-access_spider   
                                        2015-07-08  all-access_spider   
                                        2015-07-09  all-access_spider   

                                                             site country  \
Page                                    Dates                               
2NE1_zh.wikipedia.org_all-access_spider 2015-07-01  wikipedia.org      zh   
                                        2015-07-02  wikipedia.org      zh   
                                        2015-07-03  wikipedia.org      zh   
                                        2015-07-04  wikipedia.org      zh   
                                        2015-07-05  wikipedia.org      zh   
                                        2015-07-06  wikipedia.org      zh   
                                        2015-07-07  wikipedia.org      zh   
                                 

In [128]:
#Il faut encoder les données
data = pd.get_dummies(data)

In [135]:
metrics_df = pd.DataFrame(columns=['RMSE','SMAPE'])

X = data.drop('Visiteurs', axis = 1)
Y = data['Visiteurs']
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.1, random_state=0)

Xtrain = Xtrain.values
Xtest = Xtest.values
Ytrain = Ytrain.values
Ytest = Ytest.values


dtrain = xgb.DMatrix(Xtrain, label=Ytrain)
dtest = xgb.DMatrix(Xtest, label=Ytest)
param = {'max_depth':2, 'eta':1, 'objective=reg' :' tweedie'}
num_round = 2
bst = xgb.train(param, dtrain, num_round)
preds = bst.predict(dtest)

rmse = np.sqrt(mean_squared_error(Ytest, preds))
smape = smape_fast(Ytest, preds)
print("Test RMSE: %.3f" % rmse)
print("Test SMAPE: %.3f" % smape)

[15:01:09] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { objective=reg } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Test RMSE: 20.463
Test SMAPE: 83.956


In [132]:
preds

array([12.336598, 12.336598, 12.336598, ..., 12.336598, 12.336598,
       12.336598], dtype=float32)

In [1]:
metrics_df

NameError: name 'metrics_df' is not defined

In [ ]:
moyenne_RMSE = metrics_df['RMSE'].mean()
somme_RMSE = metrics_df['RMSE'].sum()
moyenne_SMAPE = metrics_df['SMAPE'].mean()

In [ ]:
print("Test moyenne RMSE: %.3f" % moyenne_RMSE)
print("Test somme RMSE: %.3f" % somme_RMSE)
print("Test moyenne SMAPE: %.3f" % moyenne_SMAPE)